In [1]:
# ! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
# Code from previous excercises

# Downloading a sample of MNIST containing just 3 and 7s supplied by fastai
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

# Creating the Training Set
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

three_tensors = [tensor(Image.open(o)) for o in threes]
seven_tensors = [tensor(Image.open(o)) for o in sevens]

stacked_threes = torch.stack(three_tensors).float()/255
stacked_sevens = torch.stack(seven_tensors).float()/255

train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
dataset = list(zip(train_x, train_y))

# Creating the Validation Set
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()]).float()/255
valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()]).float()/255

valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dataset = list(zip(valid_x, valid_y))

# Creating DataLoaders
dl = DataLoader(dataset, batch_size=256)
valid_dl = DataLoader(valid_dataset, batch_size=256)

# Defining the Loss Function
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()
    
# Calculating Gradients
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()

# Calculating Validation Accuracy
def batch_accuracy(xb,yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(),4)

# Creating an Optimiser

Because linear models have a general foundation, PyTorch provides some useful classes to make it easier to implement. The first thing we can do is replace our `linear1` function with PyTorch's `nn.Linear` module. It does the same thing as our `init_params` and `linear1` together. It contains both the *weights* and *biases* in a single class.

In [4]:
linear_model = nn.Linear(28*28, 1)

Every PyTorch module knows what parameters it has that can be trained; they are available through the `parameters` method.

In [5]:
w,b = linear_model.parameters()
w.shape, b.shape

(torch.Size([1, 784]), torch.Size([1]))

We can use this information to create an optimiser.

In [6]:
class BasicOptim:
    def __init__(self, params, lr): self.params, self.lr = list(params), lr

    def step(self, *args, **kwargs):
        for p in self.params: p.data -= p.grad.data * self.lr
    
    def zero_grad(self, *args, **kwargs):
        for p in self.params: p.grad.data.zero_()

We can create our optimiser by passsing in the model's parameters.

In [7]:
lr = 1.

opt = BasicOptim(linear_model.parameters(), lr)

Our training loop can now be simplified to:

In [8]:
def train_epoch(model):
    for xb,yb in dl:
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()

Our validation function doesn't need to change at all.

In [9]:
validate_epoch(linear_model)

0.4883

Let's put our little training loop in a function, to make things simpler.

In [10]:
def train_model(model, epochs):
    for i in range(epochs):
        train_epoch(model)
        print(validate_epoch(model), end=" ")

The results are the same as previously.


In [11]:
train_model(linear_model, 20)

0.4932 0.6655 0.855 0.9165 0.9365 0.9502 0.9565 0.9634 0.9663 0.9673 0.9702 0.9721 0.9736 0.9751 0.9761 0.977 0.9775 0.978 0.978 0.9785 

## fastai and Optimisers

To make things even easier, fastai provides the `SGD` clas which, by default, does the same thing as our `BasicOptim`.

In [12]:
linear_model = nn.Linear(28*28, 1)
opt = SGD(linear_model.parameters(), lr)
train_model(linear_model, 20)

0.4932 0.8247 0.8462 0.914 0.9355 0.9472 0.9565 0.9624 0.9658 0.9668 0.9702 0.9721 0.9736 0.9746 0.9761 0.9761 0.9775 0.9785 0.9785 0.9785 

fastai also provides `Learner.fit`, which we can use instead of `train_model`. To create a `Learner`, we first need to create a `DataLoaders`, by passing in our training and validation `DataLoader`.

In [13]:
dls = DataLoaders(dl, valid_dl)

To create a `Learner` without using an application (such as `vision_learner`), we need to pass in all the elements that we've created in this chapter.
1. The `DataLoaders`
2. The model
3. The optimisation function
4. The loss function
5. Optionally any metrics to print

In [14]:
learn = Learner(dls, nn.Linear(28*28, 1), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

We can now call `fit`.

In [15]:
learn.fit(10, lr=lr)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.637093,0.502737,0.495584,00:00
1,0.369545,0.272814,0.745339,00:00
2,0.141560,0.154224,0.861629,00:00
3,0.065627,0.097380,0.918057,00:00
4,0.037399,0.073108,0.936703,00:00
5,0.026166,0.059512,0.948970,00:00
6,0.021419,0.050921,0.957311,00:00
7,0.019218,0.045126,0.963199,00:00
8,0.018039,0.041000,0.966143,00:00
9,0.017289,0.037925,0.967125,00:00


With these classes, we can now replace our linear model with a neural network.